In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
# Read CSV file
data = pd.read_csv('melb_data.csv')

# Drop useless features
data.drop(['Address', 'Lattitude', 'Longtitude'], axis=1, inplace=True)

# Convert Date to Year and Month
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data.drop('Date', axis=1, inplace=True)

categorical_cols = ['Suburb', 'Type', 'Method', 'SellerG', 'Regionname', 'CouncilArea', 'Postcode', 'Propertycount', 'Rooms', 'Year', 'Month', 'YearBuilt']
# Replace categorical NA with -
for col in data.columns:
    if col in categorical_cols:
        if col == 'YearBuilt':
            data[col].fillna(0, inplace=True)
        else:
            data[col].fillna('-', inplace=True)
# Replace numerical NA with mean
for col in data.columns:
    if col not in categorical_cols:
        data[col].fillna(data[col].mean(), inplace=True)

data.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount,Year,Month
0,Abbotsford,2,h,1480000.0,S,Biggin,2.5,3067.0,2.0,1.0,1.0,202.0,151.96765,0.0,Yarra,Northern Metropolitan,4019.0,2016,12
1,Abbotsford,2,h,1035000.0,S,Biggin,2.5,3067.0,2.0,1.0,0.0,156.0,79.00000,1900.0,Yarra,Northern Metropolitan,4019.0,2016,2
2,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3067.0,3.0,2.0,0.0,134.0,150.00000,1900.0,Yarra,Northern Metropolitan,4019.0,2017,3
3,Abbotsford,3,h,850000.0,PI,Biggin,2.5,3067.0,3.0,2.0,1.0,94.0,151.96765,0.0,Yarra,Northern Metropolitan,4019.0,2017,3
4,Abbotsford,4,h,1600000.0,VB,Nelson,2.5,3067.0,3.0,1.0,2.0,120.0,142.00000,2014.0,Yarra,Northern Metropolitan,4019.0,2016,6


In [16]:
mappings = {}
# Convert categorical features to numeric
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(data[col])
    mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    data[col] = le.transform(data[col])
data

# Pickle mappings
import pickle
with open('mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)


In [17]:
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(data.drop('Price', axis=1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data['Price'], test_size=0.2, random_state=42)
print ("X_train.shape = ", X_train.shape)
print ("y_train.shape = ", y_train.shape)
print ("X_val.shape = ", X_test.shape)
print ("y_val.shape = ", y_test.shape)

X_train.shape =  (10864, 18)
y_train.shape =  (10864,)
X_val.shape =  (2716, 18)
y_val.shape =  (2716,)


In [18]:
def custom_loss_function(y_true, y_pred):    
    y_diff = tf.abs(tf.math.log(1+y_true) - tf.math.log(1+y_pred))    
    return tf.reduce_sum(tf.square(y_diff)) / tf.cast(tf.size(y_diff), tf.float32)

In [32]:
model2 = tf.keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model2.compile(optimizer='adam', loss=custom_loss_function, metrics=['mae'])

In [33]:
model2.fit(X_train, y_train, epochs=10, batch_size=2, verbose=1, validation_data=(X_test,y_test))

Epoch 1/10
5432/5432 [==============================] - 14s 2ms/step - loss: 21.7462 - mae: 998298.5000 - val_loss: 1.6202 - val_mae: 783682.3750
Epoch 2/10
5432/5432 [==============================] - 12s 2ms/step - loss: 0.4535 - mae: 471369.4375 - val_loss: 0.1218 - val_mae: 295836.7812
Epoch 3/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.1248 - mae: 298864.7500 - val_loss: 0.0852 - val_mae: 254125.2656
Epoch 4/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.1075 - mae: 280899.8750 - val_loss: 0.0793 - val_mae: 245586.0938
Epoch 5/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.0994 - mae: 270553.9062 - val_loss: 0.0761 - val_mae: 241639.7812
Epoch 6/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.0967 - mae: 268497.4688 - val_loss: 0.0746 - val_mae: 238782.8906
Epoch 7/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.0934 - mae: 264358.6875 - val_loss: 0.0740 - val_m

In [34]:
y_pred = model2.predict(X_test)
print(y_test.to_numpy(), y_pred)

85/85 [==============================] - 0s 1ms/step
[2600000.  620000. 1000000. ... 2450000. 1155000. 1040000.] [[1781632.5 ]
 [ 598967.25]
 [ 770237.3 ]
 ...
 [1799990.1 ]
 [1823450.4 ]
 [ 986692.7 ]]


In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(50,activation='sigmoid'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(50,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [28]:
model.compile(optimizer='adam',
              loss=custom_loss_function)
history = model.fit(X_train,y_train,
                    batch_size=32,
                    epochs=50,
                    validation_data=(X_test,y_test))

Epoch 1/50
340/340 [==============================] - 2s 4ms/step - loss: 92.0139 - val_loss: 53.1168
Epoch 2/50
340/340 [==============================] - 1s 3ms/step - loss: 41.7066 - val_loss: 33.1619
Epoch 3/50
340/340 [==============================] - 1s 3ms/step - loss: 28.0382 - val_loss: 23.5818
Epoch 4/50
340/340 [==============================] - 1s 3ms/step - loss: 20.5147 - val_loss: 17.6683
Epoch 5/50
340/340 [==============================] - 1s 3ms/step - loss: 15.5884 - val_loss: 13.6048
Epoch 6/50
340/340 [==============================] - 1s 3ms/step - loss: 12.1030 - val_loss: 10.6333
Epoch 7/50
340/340 [==============================] - 1s 3ms/step - loss: 9.5016 - val_loss: 8.3779
Epoch 8/50
340/340 [==============================] - 1s 3ms/step - loss: 7.5249 - val_loss: 6.6193
Epoch 9/50
340/340 [==============================] - 1s 3ms/step - loss: 5.9593 - val_loss: 5.2295
Epoch 10/50
340/340 [==============================] - 1s 3ms/step - loss: 4.7085 - val_

In [29]:
y_pred = model.predict(X_test).flatten()
print(y_test.to_numpy(), y_pred)

# calculate MAE of the model
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)

85/85 [==============================] - 0s 1ms/step
[2600000.  620000. 1000000. ... 2450000. 1155000. 1040000.] [1909066.2   628602.44  744822.06 ... 2048799.2  1635996.6   908537.75]
MAE: 210773.971


In [35]:
model.save('Model1.h5')
model2.save('Model2.h5')